In [1]:
import json
import os
import time
import pandas as pd
import random
import time
import pickle

import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import Dropout

from keras.backend import clear_session
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


In [2]:
with open('clean_hp_comments.json') as json_file:  #first 1mill bb comments
    data = json.load(json_file)

In [3]:

hp_tok = Tokenizer(oov_token='<unk>')
hp_tok.fit_on_texts(data)

In [4]:
len(hp_tok.word_index)

10002

In [5]:
with open('hp-v5-tok.pickle', 'wb') as handle:
    pickle.dump(hp_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [5]:


train_data = data

seqs = hp_tok.texts_to_sequences(train_data)
less_8_seqs = list(filter(lambda x: len(x) < 8, seqs))

In [6]:
len(less_8_seqs)

399347

In [7]:
MAXLEN = 7
VOCAB = int(len(hp_tok.word_index))
SEED = 42

sequences = pad_sequences(less_8_seqs, maxlen=MAXLEN, padding='pre')

print('Max Seq Len: %d' % MAXLEN)

sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=SEED)


#y_cat = to_categorical(y, num_classes=vocab_sz) #1-hot encoding 

Max Seq Len: 7


Need to optimize this later, use np arrays instead of python lists


For reference, seqs took 57s to go through 1 mill samples

In [8]:


#NN definition
model = Sequential()
model.add(Embedding(VOCAB, 300, input_length=MAXLEN-1))
model.add(Bidirectional(LSTM(500),merge_mode='concat'))
model.add(Dense(VOCAB, activation='softmax'))

print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6, 300)            3000600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000)              3204000   
_________________________________________________________________
dense_1 (Dense)              (None, 10002)             10012002  
Total params: 16,216,602
Trainable params: 16,216,602
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])


In [ ]:

model_json = model.to_json()
"""
with open("models/hp-model-v5.3.json", "w") as json_file:
    json_file.write(model_json)
"""
fpath = "weights/huffpo-v5.3-R2/best-weights.hdf5" #off of training acc
model.load_weights("weights/huffpo-v5.3/best-weights.hdf5") #prev run's weights
checkpoint = ModelCheckpoint(fpath, monitor='val_acc', verbose=2, save_best_only=True, mode='max')

BS = 32
tb = TensorBoard(log_dir="tensorboard-logs/{}".format('huffpo-modelv5.3-bilstm(R2)'))
callback_lst = [checkpoint, tb]
#steps_per_epoch is num of batches that make up 1 epoch, defaults to size of train set
model.fit(X_train,y_train,batch_size=BS, validation_split=.01, epochs=200, callbacks=callback_lst, verbose=1)



Train on 336049 samples, validate on 3395 samples
Epoch 1/200
336049/336049 [==============================] - 732s 2ms/step - loss: 3.7805 - acc: 0.3515 - val_loss: 4.6789 - val_acc: 0.3146

Epoch 00001: val_acc improved from -inf to 0.31458, saving model to weights/huffpo-v5.3-R2/best-weights.hdf5
Epoch 2/200
336049/336049 [==============================] - 751s 2ms/step - loss: 3.2887 - acc: 0.3971 - val_loss: 4.8406 - val_acc: 0.3049

Epoch 00002: val_acc did not improve from 0.31458
Epoch 3/200
 16192/336049 [>.............................] - ETA: 12:56 - loss: 2.6286 - acc: 0.5012